In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]

In [2]:
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [78]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="mixtral-8x7b-32768")

In [86]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=5)
search.invoke("tell me about 2025 LA wildfire updates")

[{'url': 'https://recovery.lacounty.gov/2025/01/17/wildfire-update-january-17-2025/',
  'content': 'Wildfire Update – January 17, 2025 · Structures Threatened: 14,518 · Structures Damaged: 805 · Structures Destroyed: 7,193 · Structures Threatened:'},
 {'url': 'https://www.allhandsandhearts.org/storm-tracker/los-angeles-california-wildfires-january-2025/',
  'content': 'In total, the Los Angeles wildfires continue to rage over 35,000 acres burned and over 10,000 structures destroyed (CAL FIRE). January 10, 2025 - 14:30 (EST) -'},
 {'url': 'https://en.wikipedia.org/wiki/January_2025_Southern_California_wildfires',
  'content': 'From January\xa07 to 31, 2025, a series of 14 destructive wildfires affected the Los Angeles metropolitan area and San Diego County in California, United States.[5] The fires were exacerbated by drought conditions, low humidity, a buildup of vegetation from the previous winter, and hurricane-force Santa Ana winds, which in some places reached 100 miles per hour (1

In [87]:
tools = [search]

In [88]:
tools

[TavilySearchResults()]

In [93]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# Assuming tools and llm are already defined
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Take user input
user_query = input("Enter your query: ")

# Add formatting constraint to user query
formatted_query = (
    f"{user_query}\n\n"
    "Please provide your answer in clear bullet points (• or -), "
    "limit the total response to around 150 words. "
    "Keep it concise, informative, and structured."
)

# Run agent with formatted query
response = agent.run(formatted_query)
print("\nResponse:\n")
print(response)




> Entering new AgentExecutor chain...
I need to find information about Rachin Ravindra's eye injury.

Action: tavily_search_results_json

Action Input: Rachin Ravindra eye injury

Observation: [{'url': 'https://www.instagram.com/upscworldofficial/p/DF0jsB3tbBc/', 'content': 'New Zealand cricketer Rachin Ravindra suffered an injury near his eye while attempting a catch against Pakistan.'}, {'url': 'https://sports.ndtv.com/cricket/pakistan-puts-blame-back-on-rachin-ravindra-over-nasty-head-injury-despite-floodlights-issue-7683652', 'content': 'New Zealand cricketer Rachin Ravindra incurred a nasty head injury during an ODI match against Pakistan at the Gaddafi Stadium in Lahore.'}, {'url': 'https://www.arabnews.com/node/2589539/sport', 'content': "LAHORE: New Zealand's Rachin Ravindra suffered a sickening blow to the head after he appeared to misjudge a catch during his team's ODI"}, {'url': 'https://www.indiatoday.in/sports/story/cricket-rachin-ravindra-gets-brutally-injured-leaves-fi

In [82]:
from langgraph.prebuilt.chat_agent_executor import LangGraphInput


<class 'pydantic.v1.main.LangGraphInput'>


In [60]:
search_results = search.invoke("2025 LA wildfire updates")
web_content = "\n\n".join([res.get('content', '').strip() for res in search_results if res.get('content')])

summary = llm.invoke(f"""You are a wildfire monitoring assistant.

Below is the latest web content about Los Angeles wildfires. Summarize key points.

Web Content:
{web_content}
""")

print(summary)


content='In January 2025, a series of 14 destructive wildfires affected the Los Angeles metropolitan area and San Diego County in California, causing loss of lives, destruction of homes and businesses, and displacement of families. The wildfires were caused by drought conditions, low humidity, accumulation of vegetation, and hurricane-force Santa Ana winds, which reached up to 100 miles per hour. California Governor Gavin Newsom allocated resources, including 65 fire engines, 7 helicopters, 7 water tenders, and 109 workers, to combat the wildfires. Mayor of Los Angeles Karen Bass warned residents of the harsh windstorm and advised them to avoid downed power lines. Due to the disaster, businesses affected by the fires can receive tax relief and extensions, and victims can restore their homes and businesses faster as permit and review requirements under the California Environmental Quality Act and the California Coastal Act were suspended. On January 8, California received a Major Disast

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [94]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [16]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "001"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UEgG16mvYiSPrP2eCTKcIkYZ', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c91979be-03ad-4840-98df-fba9130d4d43-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_UEgG16mvYiSPrP2eCTKcIkYZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [ToolMe

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m0tbNUOYWsEkJ32B9ZsATvCw', 'function': {'arguments': '{"query":"top stars of the winning team"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 91, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d2de8a4-0276-41e9-9be3-b5166f83ffc4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top stars of the winning team'}, 'id': 'call_m0tbNUOYWsEkJ32B9ZsATvCw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 24, 'total_tokens': 115})]}}
----
{'tools': {'messages': [

In [28]:
config = {"configurable": {"thread_id": "001"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned the soccer team "Manchester United" in our previous conversation. Would you like me to provide more information about them?', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 89, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9fdab43-09da-4e58-9b4e-caff79772e3d-0', usage_metadata={'input_tokens': 89, 'output_tokens': 27, 'total_tokens': 116})]}}
----


In [115]:
from langchain_groq import ChatGroq
from langchain.agents import Tool, load_tools
from langchain.agents.agent_types import AgentType
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage,AIMessage
import os
import os



# Step 3: Setup memory
memory = MemorySaver()
thread_id = "001"  # Unique session/thread ID

# Step 4: Simulate initial conversation in memory (now includes 'id' key)
initial_checkpoint = {
    "id": "checkpoint_001",  # <-- REQUIRED ID
    "messages": [
        HumanMessage(content="Who won Champions Trophy 2025 final?"),
        AIMessage(content="India won the 2025 Champions Trophy final by defeating England.")
    ]
}

# Save initial memory
memory.put(
    config={"configurable": {"thread_id": thread_id}},
    checkpoint=initial_checkpoint,
    metadata={}
)

# Step 5: Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# Step 6: Fetch memory checkpoint
checkpoint = memory.get(config={"configurable": {"thread_id": thread_id}})
context = ""
if checkpoint and "messages" in checkpoint:
    context = "\n".join([f"{type(msg).__name__}: {msg.content}" for msg in checkpoint["messages"]])

# Step 7: Ask new user query
user_query = input("Enter your query: ")

# Step 8: Format the query for bullet-point style response
formatted_query = (
    f"Conversation so far:\n{context}\n\n"
    f"Now user asked: {user_query}\n"
    "Respond in bullet points (• or -), keep it concise and under ~150 words."
)

# Step 9: Get agent response
response = agent.run(formatted_query)
print("\nResponse:\n")
print(response)

# Step 10: Save updated memory
updated_checkpoint = {
    "id": "checkpoint_002",  # New unique ID for new checkpoint
    "messages": checkpoint["messages"] + [
        HumanMessage(content=user_query),
        AIMessage(content=response)
    ]
}

memory.put(
    config={"configurable": {"thread_id": thread_id}},
    checkpoint=updated_checkpoint,
    metadata={}
)



> Entering new AgentExecutor chain...
Thought: The user seems to be interested in India's cricket trophy wins. I should provide a brief and concise response summarizing India's recent cricket trophy victories.

Action: tavily_search_results_json
Action Input: "India cricket trophy wins 2020-2025"

Observation: [{'url': 'https://www.youtube.com/watch?v=2tgLmSrM2oQ', 'content': 'India is the ICC Champions Trophy 2025 winner. During the grand final in Dubai on Sunday, India beat New Zealand by four wickets to clinch'}, {'url': 'https://indianexpress.com/article/sports/cricket/champions-trophy-winners-list-india-near-record-third-title-in-2025-final-against-new-zealand-9877709/', 'content': 'Champions Trophy winners list: India win record 3rd title in 2025 final against New Zealand | Cricket News - The Indian Express.'}, {'url': 'https://sportstar.thehindu.com/cricket/champions-trophy/india-full-list-of-icc-title-wins-results-by-year-champions-trophy-2025-final-ind-vs-new-zealand/article

{'configurable': {'thread_id': '001', 'thread_ts': 'checkpoint_002'}}